In [7]:
# Importing necessary libraries.
import os
import numpy as np
import matplotlib.pyplot as plt
import random
import tensorflow as tf
from tensorflow.data import Dataset
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow import keras
import cv2
import pandas as pd

In [8]:
# Checking the tensorflow version. It is important to know the version as it comes handy during deployment. 
tf.__version__

'2.4.1'

In [9]:
# Global variables.
IMSIZE = 49
BATCH_SIZE = 8
THREADS = 32
SEED = 125
EPSILON = 0.001
N_CLASSES = 43
PATH = 'Training_Data'

In [10]:
# Initializing variables.
filenames = []
labels = []
train_X = []
train_y = []
validation_X = []
validation_y = []

In [11]:
# Creating one hot vector for each class.
Categories = np.eye(43)

In [12]:
# Creating the list of images in every class.
for dirname in os.listdir(PATH):
    for fname in os.listdir(os.path.join(PATH, dirname)):
        filenames.append(str(os.path.join(PATH, dirname, fname)))

In [13]:
# Shuffeling the data
random.seed(SEED)
random.shuffle(filenames)

In [14]:
print(filenames[0])

Training_Data\00035\aug_4599_00010_00011..bmp


In [15]:
# Creating the labels vector.
for filename in filenames:
    labels.append(Categories[int(filename.split('\\')[-2])])

In [16]:
# Split data into training and validation.
dataset = list(zip(filenames, labels))
print("dataset length", len(dataset))
print("train dataset length", int(0.8*len(dataset)))
train_dataset = dataset[:int(len(dataset)*0.8)]
validation_dataset = dataset[int(len(dataset)*0.8):]

dataset length 429546
train dataset length 343636


In [17]:
# Creatig the vector of training images and labels.
for feature, label in train_dataset:
    train_X.append(feature)
    train_y.append(label)

In [18]:
# Creatig the vector of validation images and labels.
for feature, label in validation_dataset:
    validation_X.append(feature)
    validation_y.append(label)

In [19]:
# Image preprocessing function.
def parse_function(filename, label):
    image_string = tf.io.read_file(filename)
    image = tf.image.decode_bmp(image_string, channels=3)
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, [IMSIZE, IMSIZE])
    return image, label

In [20]:
# tensorflow function to create a datapipeline and load training images to RAM/GPU.
train_data = Dataset.from_tensor_slices((train_X, train_y))
train_data = train_data.shuffle(len(train_X))
train_data = train_data.map(parse_function, num_parallel_calls=THREADS)
train_data = train_data.batch(BATCH_SIZE)
train_data = train_data.prefetch(1)

In [21]:
# tensorflow function to create a datapipeline and load validation images to RAM/GPU.
validation_data = Dataset.from_tensor_slices((validation_X, validation_y))
validation_data = validation_data.shuffle(len(train_X))
validation_data = validation_data.map(parse_function, num_parallel_calls=THREADS)
validation_data = validation_data.batch(BATCH_SIZE)
validation_data = validation_data.prefetch(1)

In [22]:
# Creating the model.
def Model(output_classes):
    
    model = Sequential()

    model.add(Conv2D(32, (3,3), strides=(1,1), padding='same', activation = 'relu', 
                     input_shape = (IMSIZE, IMSIZE, 3)))
    model.add(MaxPooling2D(pool_size=(2,2)))

    model.add(Conv2D(32, (3,3), strides=(1,1), padding='same', 
                     activation = 'relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))

    model.add(Conv2D(64, (3,3), strides=(1,1), padding='same', 
                     activation = 'relu'))
    model.add(Conv2D(64, (3,3), strides=(1,1), padding='same', 
                     activation = 'relu'))
    model.add(Conv2D(128, (3,3), strides=(1,1), padding='same', 
                     activation = 'relu'))

    model.add(MaxPooling2D(pool_size=(2,2)))

    model.add(Flatten())
    model.add(Dense(128))
    model.add(Dense(64))
    model.add(Dense(32))   #newly added
    model.add(Dense(output_classes))
    model.add(Activation("softmax"))
    
    return model

In [23]:
# Initialize model.
deepNet = Model(N_CLASSES)

In [24]:
# Create model checkpoints to save the model weights and bias after every epoch.
save_status = [tf.keras.callbacks.ModelCheckpoint(filepath='Saved_ephocs_v3.0\\deepNet.{epoch:02d}-{val_loss:.4f}.h5')]

In [25]:
# Compile model.
deepNet.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=3e-4), 
              metrics=['accuracy'])

In [26]:
# Train model.
model_design = deepNet.fit(train_data, validation_data=validation_data, 
                         epochs=50, callbacks=save_status)

Epoch 1/50
42955/42955 [==============================] - 1493s 35ms/step - loss: 0.7010 - accuracy: 0.7962 - val_loss: 0.0666 - val_accuracy: 0.9789
Epoch 2/50
42955/42955 [==============================] - 1100s 26ms/step - loss: 0.0597 - accuracy: 0.9817 - val_loss: 0.0623 - val_accuracy: 0.9816
Epoch 3/50
42955/42955 [==============================] - 869s 20ms/step - loss: 0.0402 - accuracy: 0.9880 - val_loss: 0.0425 - val_accuracy: 0.9886
Epoch 4/50
42955/42955 [==============================] - 868s 20ms/step - loss: 0.0319 - accuracy: 0.9905 - val_loss: 0.0385 - val_accuracy: 0.9893
Epoch 5/50
42955/42955 [==============================] - 874s 20ms/step - loss: 0.0288 - accuracy: 0.9918 - val_loss: 0.0500 - val_accuracy: 0.9874
Epoch 6/50
42955/42955 [==============================] - 877s 20ms/step - loss: 0.0266 - accuracy: 0.9925 - val_loss: 0.0441 - val_accuracy: 0.9897
Epoch 7/50
42955/42955 [==============================] - 876s 20ms/step - loss: 0.0249 - accuracy: 0.99

In [ ]:
# Visulize the change in loss value after every epoch.
plt.figure(figsize=(8,6), facecolor='#848484', edgecolor='black')
plt.title('Loss', fontsize= 24)
plt.plot(model_design.history['loss'], label = 'train', color='#2E86C1')
plt.plot(model_design.history['val_loss'], label = 'validation', color='#1D8348')
plt.legend(prop={'size': 18})
plt.xticks(np.arange(0, 55, 5),fontsize=12)
plt.yticks(np.arange(0, 2, 0.05),fontsize=12)
plt.ylabel('Loss', fontsize=18)
plt.xlabel('Epochs', fontsize=18)
plt.grid()
plt.show()

In [ ]:
# Scaling accuracy to the range of 100%.
train_accuracy_list = np.array(model_design.history['accuracy'])*100
val_accuracy_list = np.array(model_design.history['val_accuracy'])*100

In [ ]:
# Visulize the change in accuracy value after every epoch.
plt.figure(figsize=(16,9), facecolor='#848484', edgecolor='black')
plt.title('Accuracy', fontsize= 24, fontname='Times New Roman')
plt.plot(train_accuracy_list, label='train', color='#2E86C1')
plt.plot(val_accuracy_list, label='validation', color='#1D8348')
plt.legend(prop={'family':'Times New Roman', 'size':18})
plt.xticks(np.arange(0, 110, 10),fontsize=12, fontname='Times New Roman')
plt.yticks(np.arange(80, 101, 2),fontsize=12, fontname='Times New Roman')
plt.ylabel('Accuracy', fontsize=18, fontname='Times New Roman')
plt.xlabel('Epochs', fontsize=18, fontname='Times New Roman')
plt.grid()
plt.show()